## Import Libraries 

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
pd.options.display.max_columns = 999

## Read Data 

In [2]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')
meal_info = pd.read_csv('../data/meal_info.csv')
fulfillment_center = pd.read_csv('../data/fulfilment_center_info.csv')
submission = pd.read_csv('../data/sample_submission.csv')

In [3]:
train.head()

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders
0,1379560,1,55,1885,136.83,152.29,0,0,177
1,1466964,1,55,1993,136.83,135.83,0,0,270
2,1346989,1,55,2539,134.86,135.86,0,0,189
3,1338232,1,55,2139,339.50,437.53,0,0,54
4,1448490,1,55,2631,243.50,242.50,0,0,40


In [4]:
test.head()

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured
0,1028232,146,55,1885,158.11,159.11,0,0
1,1127204,146,55,1993,160.11,159.11,0,0
2,1212707,146,55,2539,157.14,159.14,0,0
3,1082698,146,55,2631,162.02,162.02,0,0
4,1400926,146,55,1248,163.93,163.93,0,0


In [5]:
meal_info.head()

,meal_id,category,cuisine
0,1885,Beverages,Thai
1,1993,Beverages,Thai
2,2539,Beverages,Thai
3,1248,Beverages,Indian
4,2631,Beverages,Indian


In [6]:
fulfillment_center.head()

,center_id,city_code,region_code,center_type,op_area
0,11,679,56,TYPE_A,3.7
1,13,590,56,TYPE_B,6.7
2,124,590,56,TYPE_C,4.0
3,66,648,34,TYPE_A,4.1
4,94,632,34,TYPE_C,3.6


In [8]:
print("Number of rows in training set:", train.shape[0])
print("Number of columns in training set:", train.shape[1])
print("Number of rows in test set:", test.shape[0])
print("Number of columns in test set:", test.shape[1])

Number of rows in training set: 456548
Number of columns in training set: 9
Number of rows in test set: 32573
Number of columns in test set: 8


## Merge Data 

In [9]:
train = pd.merge(train, meal_info, how='left', on='meal_id')
test = pd.merge(test, meal_info, how='left', on='meal_id')
train = pd.merge(train, fulfillment_center, how='left', on='center_id')
test = pd.merge(test, fulfillment_center, how='left', on='center_id')

In [10]:
train.shape

(456548, 15)

In [11]:
test.shape

(32573, 14)

In [13]:
train.head()

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders,category,cuisine,city_code,region_code,center_type,op_area
0,1379560,1,55,1885,136.83,152.29,0,0,177,Beverages,Thai,647,56,TYPE_C,2.0
1,1466964,1,55,1993,136.83,135.83,0,0,270,Beverages,Thai,647,56,TYPE_C,2.0
2,1346989,1,55,2539,134.86,135.86,0,0,189,Beverages,Thai,647,56,TYPE_C,2.0
3,1338232,1,55,2139,339.50,437.53,0,0,54,Beverages,Indian,647,56,TYPE_C,2.0
4,1448490,1,55,2631,243.50,242.50,0,0,40,Beverages,Indian,647,56,TYPE_C,2.0


In [14]:
# set id as index
train.set_index('id', inplace=True)
test.set_index('id', inplace=True)

In [15]:
train.head()

,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders,category,cuisine,city_code,region_code,center_type,op_area
id,,,,,,,,,,,,,,
1379560,1,55,1885,136.83,152.29,0,0,177,Beverages,Thai,647,56,TYPE_C,2.0
1466964,1,55,1993,136.83,135.83,0,0,270,Beverages,Thai,647,56,TYPE_C,2.0
1346989,1,55,2539,134.86,135.86,0,0,189,Beverages,Thai,647,56,TYPE_C,2.0
1338232,1,55,2139,339.50,437.53,0,0,54,Beverages,Indian,647,56,TYPE_C,2.0
1448490,1,55,2631,243.50,242.50,0,0,40,Beverages,Indian,647,56,TYPE_C,2.0


In [16]:
train['emailer_for_promotion'].value_counts()

0    419498
1     37050
Name: emailer_for_promotion, dtype: int64

In [17]:
train['homepage_featured'].value_counts()

0    406693
1     49855
Name: homepage_featured, dtype: int64

## Feature Engineering 

In [21]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 456548 entries, 1379560 to 1361984
Data columns (total 14 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   week                   456548 non-null  int64  
 1   center_id              456548 non-null  int64  
 2   meal_id                456548 non-null  int64  
 3   checkout_price         456548 non-null  float64
 4   base_price             456548 non-null  float64
 5   emailer_for_promotion  456548 non-null  int64  
 6   homepage_featured      456548 non-null  int64  
 7   num_orders             456548 non-null  int64  
 8   category               456548 non-null  object 
 9   cuisine                456548 non-null  object 
 10  city_code              456548 non-null  int64  
 11  region_code            456548 non-null  int64  
 12  center_type            456548 non-null  object 
 13  op_area                456548 non-null  float64
dtypes: float64(3), int64(8), obje

In [22]:
# convert numerical columns to categorical

train['center_id'] = train['center_id'].astype('category')
test['center_id'] = test['center_id'].astype('category')

train['emailer_for_promotion'] = train['emailer_for_promotion'].astype('category')
test['emailer_for_promotion'] = test['emailer_for_promotion'].astype('category')

train['homepage_featured'] = train['homepage_featured'].astype('category')
test['homepage_featured'] = test['homepage_featured'].astype('category')

train['city_code'] = train['city_code'].astype('category')
test['city_code'] = test['city_code'].astype('category')

train['region_code'] = train['region_code'].astype('category')
test['region_code'] = test['region_code'].astype('category')

In [23]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 456548 entries, 1379560 to 1361984
Data columns (total 14 columns):
 #   Column                 Non-Null Count   Dtype   
---  ------                 --------------   -----   
 0   week                   456548 non-null  int64   
 1   center_id              456548 non-null  category
 2   meal_id                456548 non-null  int64   
 3   checkout_price         456548 non-null  float64 
 4   base_price             456548 non-null  float64 
 5   emailer_for_promotion  456548 non-null  category
 6   homepage_featured      456548 non-null  category
 7   num_orders             456548 non-null  int64   
 8   category               456548 non-null  object  
 9   cuisine                456548 non-null  object  
 10  city_code              456548 non-null  category
 11  region_code            456548 non-null  category
 12  center_type            456548 non-null  object  
 13  op_area                456548 non-null  float64 
dtypes: category(5

In [24]:
train.head()

,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders,category,cuisine,city_code,region_code,center_type,op_area
id,,,,,,,,,,,,,,
1379560,1,55,1885,136.83,152.29,0,0,177,Beverages,Thai,647,56,TYPE_C,2.0
1466964,1,55,1993,136.83,135.83,0,0,270,Beverages,Thai,647,56,TYPE_C,2.0
1346989,1,55,2539,134.86,135.86,0,0,189,Beverages,Thai,647,56,TYPE_C,2.0
1338232,1,55,2139,339.50,437.53,0,0,54,Beverages,Indian,647,56,TYPE_C,2.0
1448490,1,55,2631,243.50,242.50,0,0,40,Beverages,Indian,647,56,TYPE_C,2.0


### Week 

In [28]:
top10_weeks = train.groupby('week')['num_orders'].sum().reset_index().sort_values(by='num_orders', ascending=False)[:10]
top10_weeks

,week,num_orders
47,48,1303457
4,5,1198675
59,60,1131459
52,53,1108236
99,100,1088162
68,69,1075188
108,109,1068318
51,52,1046811
31,32,1034202
70,71,1015726


In [33]:
week_orders = train.groupby('week')['num_orders'].sum().reset_index()
week_orders

,week,num_orders
0,1,792261
1,2,787084
2,3,695262
3,4,743529
4,5,1198675
...,...,...
140,141,776240
141,142,739661
142,143,759926
143,144,801197


In [35]:
fig = px.line(week_orders, x='week', y='num_orders', title='Weekly Orders')
fig.show()

The highest numbers of orders is in the week 48, 5, and 60. The lowest number of orders are in the week 62. 

In [39]:
weekly_sales = train.groupby('week')['checkout_price'].sum().reset_index().rename(columns={'checkout_price':'total_sales'})
weekly_sales

,week,total_sales
0,1,965809.26
1,2,938092.43
2,3,948560.28
3,4,953833.69
4,5,901141.58
...,...,...
140,141,1133461.40
141,142,1124552.90
142,143,1131454.15
143,144,1113608.58


In [40]:
# which week has the highest sales 
weekly_sales.sort_values(by='total_sales', ascending=False)

,week,total_sales
117,118,1140144.83
104,105,1138084.07
140,141,1133461.40
116,117,1132047.49
119,120,1131995.17
...,...,...
8,9,923631.46
61,62,921837.04
7,8,919479.78
5,6,912920.01


The highest sales are in the week 117, 104 and 140. And lowest sales are in the week 4, 5 and 7. Although the total number of orders in week 5 is very high,but the total sales amount is low compared to other weeks. 

In [42]:
fig = px.line(weekly_sales, x='week', y='total_sales', title='weekly sales')
fig.show()

As the week increases the total sales per week also increases. But there are few weeks like week 62 and 103 when sales dip. The number of orders in week 62 is also very low. 

### Fulfillment Center 

In [45]:
train.groupby('center_id')['num_orders'].sum().reset_index().sort_values(by='num_orders', ascending=False)

,center_id,num_orders
2,13,4296545
18,43,3920294
0,10,3381018
21,52,3125835
63,137,3117478
...,...,...
35,74,691044
73,162,680255
68,149,637527
16,41,497338


The highest number of orders are from center 13, 43 and 10. The lowest number of orders are from center 91, 41 and 149. 

## Category 

In [49]:
category_orders = train.groupby('category')['num_orders'].sum().reset_index().sort_values(by='num_orders', ascending=False)
category_orders

,category,num_orders
0,Beverages,40480525
8,Rice Bowl,20874063
10,Sandwich,17636782
9,Salad,10944336
7,Pizza,7383720
5,Other Snacks,4766293
13,Starters,4649122
3,Extras,3984979
11,Seafood,2715714
2,Desert,1940754


In [55]:
fig = px.bar(category_orders, x='num_orders', y='category', 
             color_discrete_sequence=['seagreen'])
fig.update_layout(yaxis=dict(autorange="reversed"))
fig.show()

In [64]:
category_sales = train.groupby('category')['checkout_price'].sum().reset_index().sort_values(by='checkout_price',ascending=False)
category_sales.rename(columns={'checkout_price':'total_sales'}, inplace=True)
category_sales

,category,total_sales
0,Beverages,27274335.97
7,Pizza,19352729.40
11,Seafood,16434870.57
2,Desert,12819059.27
6,Pasta,10957496.95
8,Rice Bowl,10608117.08
10,Sandwich,9654023.75
1,Biryani,9284651.88
13,Starters,8331741.86
9,Salad,8309396.40


In [65]:
fig = px.bar(category_sales, x='total_sales', y='category', color_discrete_sequence=['seagreen'])
fig.show()

Although the total number of sales of pizza, seafood, Desert and Pasta are very low, the total sales for these products are very high. The total sales are in the top 5. So we should promote these products more on the homepage. 

## cuisine

In [68]:
cuisine_orders = train.groupby('cuisine')['num_orders'].sum().reset_index().sort_values(by='num_orders', ascending=False)
cuisine_orders

,cuisine,num_orders
2,Italian,44172832
3,Thai,32677670
1,Indian,25792544
0,Continental,16914439


In [69]:
fig = px.bar(cuisine_orders, x='num_orders', y='cuisine',
            color_discrete_sequence=['seagreen'])
fig.show()

In [76]:
cuisine_sales = train.groupby('cuisine')['checkout_price'].sum().reset_index().sort_values(by='checkout_price', ascending=False)
cuisine_sales.rename(columns={'checkout_price':'total_sales'}, inplace=True)
cuisine_sales

,cuisine,total_sales
0,Continental,53040041.70
1,Indian,38374772.38
2,Italian,35049986.61
3,Thai,25218219.49


In [77]:
fig = px.bar(cuisine_sales, x='total_sales', y='cuisine', color_discrete_sequence=['seagreen'])
fig.show()

Although total number of orders from continental cuisine is very low,but the total sales from continental cuisine is very high. 